In [89]:
# Imports
import numpy as np
from pathlib import Path
import random
import os

# Dynamické zjištění kořenového adresáře projektu
try:
    # Toto funguje, pokud je notebook spuštěn jako soubor (např. ve VS Code z file exploreru)
    # nebo pokud __file__ je definováno
    current_script_path = Path(__file__).resolve()
    # Předpokládáme, že notebook je ve složce 'notebooks', která je přímo v kořeni projektu
    project_root = current_script_path.parent.parent 
except NameError:
    # __file__ není definováno (např. interaktivní spuštění buňky nebo starší Jupyter)
    # Použijeme os.getcwd() a zkontrolujeme, zda jsme ve složce 'notebooks'
    # nebo už v kořeni projektu.
    cwd = Path(os.getcwd())
    if cwd.name == "notebooks" and (cwd.parent / "data").is_dir() and (cwd.parent / "scripts").is_dir():
        project_root = cwd.parent
    elif (cwd / "data").is_dir() and (cwd / "scripts").is_dir():
        project_root = cwd # Vypadá to, že CWD je už kořen projektu
    else:
        # Nouzové řešení, pokud struktura není rozpoznána - může vyžadovat manuální úpravu
        project_root = Path(".").resolve() # Použije aktuální CWD jako kořen
        print(f"Warning: Could not reliably determine project root. Using CWD: {project_root}")
        print("Please ensure this notebook is run from the project's root directory or the 'notebooks' subdirectory.")

print(f"Project root determined as: {project_root}")

# Importy vašich modulů z knihovny subcipher
from subcipher.mh import crack
from subcipher.io import export_result
from subcipher.codec import encrypt, decrypt
from subcipher.text_utils import clean_text
from subcipher.alphabet import ALPHABET

print("Subcipher modules imported successfully.")

# --- Nyní definujte cesty k souborům relativně k project_root ---

# Cesta k referenční matici (vytvořené z krakatit.txt)
model_path = project_root / "data" / "model" / "reference_tm.npy"
print(f"Path to reference model set to: {model_path}")

# Cesta k šifrovanému textu od učitele (nebo vašemu testovacímu)
# Ujistěte se, že tento soubor existuje na dané cestě!
ciphertext_file_path = project_root / "data" / "test" / "text_1000_sample_1_ciphertext.txt" 
print(f"Path to ciphertext file set to: {ciphertext_file_path}")

# Adresář pro export výsledků (bude vytvořen, pokud neexistuje)
output_dest_dir_attack = project_root / "exports" / "notebook_attack_results" # Přejmenoval jsem proměnnou pro jasnost
print(f"Output directory for attack results set to: {output_dest_dir_attack}")

#---------------------------------------------------------------------
# Následující buňky vašeho notebooku by pak měly používat proměnné:
# model_path
# ciphertext_file_path
# output_dest_dir_attack
# ALPHABET
# a funkce crack, export_result, encrypt, decrypt, clean_text
#---------------------------------------------------------------------

Project root determined as: c:\Git\DeShiftr
Subcipher modules imported successfully.
Path to reference model set to: c:\Git\DeShiftr\data\model\reference_tm.npy
Path to ciphertext file set to: c:\Git\DeShiftr\data\test\text_1000_sample_1_ciphertext.txt
Output directory for attack results set to: c:\Git\DeShiftr\exports\notebook_attack_results


In [90]:
# Load the reference transition matrix
# Proměnná model_path je již definována v první buňce a neměla by se zde přepisovat.
if not model_path.exists():
    print(f"Model file not found: {model_path}")
    print("Please ensure you have built and saved the reference_tm.npy first (e.g., using 02_bigram_model.ipynb or build_bigram_model.py).")
    raise FileNotFoundError(f"Model file not found: {model_path}. Please generate it first.")
else:
    tm_ref = np.load(model_path)
    print(f"Successfully loaded reference transition matrix from '{model_path}'. Shape: {tm_ref.shape}")

Successfully loaded reference transition matrix from 'c:\Git\DeShiftr\data\model\reference_tm.npy'. Shape: (27, 27)


In [91]:
# Prepare Ciphertext
# Proměnná ciphertext_file_path je již definována v první buňce a neměla by se zde přepisovat.

# Inicializace proměnných pro demo text, pokud by hlavní soubor nebyl nalezen
true_plaintext_demo = None
true_key_demo = None

if ciphertext_file_path.exists() and ciphertext_file_path.is_file():
    with open(ciphertext_file_path, 'r', encoding='utf-8') as f:
        raw_ciphertext = f.read()
    print(f"Loaded ciphertext from file: {ciphertext_file_path}. Length: {len(raw_ciphertext)}")
else:
    print(f"Ciphertext file {ciphertext_file_path} not found or is not a file. Using a demo ciphertext.")
    true_plaintext_demo = "TOTO_JE_DEMONSTRACNI_TEXT_KTERY_BUDE_ZASIFROVAN_A_POTOM_PROLOMEN_POMOCI_METROPOLIS_HASTINGS_ALGORITMU_SNAD_TO_VYJDE_DOBRE"
    key_list_demo = list(ALPHABET)
    random.seed(42) # Přidáno pro reprodukovatelnost demo klíče
    random.shuffle(key_list_demo)
    true_key_demo = "".join(key_list_demo)
    raw_ciphertext = encrypt(true_plaintext_demo, true_key_demo)
    print(f"Using demo ciphertext. True key for demo: {true_key_demo}")
    print(f"Demo Plaintext: {true_plaintext_demo}")
    print(f"Demo Ciphertext (raw): {raw_ciphertext[:100]}...")


# Clean the ciphertext
ciphertext_to_crack = clean_text(raw_ciphertext)
print(f"Ciphertext to crack (cleaned, length {len(ciphertext_to_crack)}): {ciphertext_to_crack[:100]}...")

Loaded ciphertext from file: c:\Git\DeShiftr\data\test\text_1000_sample_1_ciphertext.txt. Length: 1000
Ciphertext to crack (cleaned, length 1000): ABM_DEAOMARDHMAVA_VNAERDALD_UAOMAZDNYPAA_VZHBDSVANDAYVWAWIOPABCKVBMARDLMABSDBMAYDOPAXDAWMRDZACYVSAND...


In [ ]:
# Perform the cryptanalysis using crack()
iters = 40000  
temp = 1.0 # Můžete experimentovat s touto hodnotou
seed = None  # Nastavení seedu pro reprodukovatelné výsledky M-H, None pro náhodné

print(f"\nStarting cryptanalysis with {iters} iterations, temp={temp}, seed={seed}...")
found_key, decrypted_plaintext, best_ll = crack(
    ciphertext_to_crack,
    tm_ref,
    iters=iters,
    temp=temp,
    seed=seed
)

print(f"\nCryptanalysis finished.")
print(f"Best Log-Likelihood: {best_ll:.4f}")
print(f"Found Key:           {found_key}")
print(f"\nDecrypted Plaintext (first 500 characters):")
print(decrypted_plaintext[:500])

# Kontrola pro demo text, pokud byl použit
if true_plaintext_demo: # Zkontroluje, zda byla proměnná true_plaintext_demo nastavena
    matches = sum(p == t for p, t in zip(decrypted_plaintext, true_plaintext_demo))
    accuracy = matches / len(true_plaintext_demo) if len(true_plaintext_demo) > 0 else 0
    print(f"\n--- Demo Ciphertext Check ---")
    print(f"True Key for Demo:    {true_key_demo}") # true_key_demo je definován pouze v demo bloku
    print(f"Accuracy with true demo plaintext: {accuracy:.2%}")
    if found_key == true_key_demo:
        print("Successfully recovered the true demo key!")
    else:
        print("Did not recover the exact true demo key.")


Starting cryptanalysis with 100000 iterations, temp=1.1, seed=None...

Cryptanalysis finished.
Best Log-Likelihood: -11086.9912
Found Key:           AZ_ODKGCUXWVENMLTPRYIBFJHSQ

Decrypted Plaintext (first 500 characters):
AVOCEMADOASEYOALACLNAMSEAPECIADOABENTRAACLBYVEZLANEATLKAKUDRAVHFLVOASEPOAVZEVOATEDRAJEAKOSEBAHTLZANEATINEAHOKRVZLAYZLVOUATEDRAJEAKOSEBAOTEVFEZADVIFKLAVRNKOBIZACAVOCUALAHONTLVIZANEAHFEDAKOZLAJEDAFEKZABYFLHTIVEAHOJEDENAHFENEAMSEAUJEZLANAVOCEMADVLAKFOKRACHETAHOJDAMUNIMEADLZADOVECUATEALNHOSAPZICAKAYFLSIBIMAKLMABYBENACHLTKRANKFIHEZACUPRACHLTKRANATEPOUANEAMSOUASESIALSIADOHFEDUALSIACHLTKRABOHLKAMIASEFOCUMINAMUNIMATOAUDEZLTALPRNAVIDEZALPRAPRZOAJINTOACEAJNEMATEAMEZLAFLDLAMRNZINACEAPRBYAMOYZLAJENTEAJEDSOU


In [93]:
# Buňka 5: Export výsledků

# Znovu definujeme/zajistíme existenci output_dest_dir_for_attack pro případ,
# že by tato buňka byla spuštěna samostatně po restartu kernelu,
# ale project_root musí být známý z Buňky 1.
# Pokud Buňka 1 běžela, tato proměnná už existuje.
# Pokud by neběžela, je třeba zajistit definici project_root i zde,
# ale pro jednoduchost předpokládáme, že Buňka 1 běžela.
if 'project_root' not in locals() or not project_root:
    print("Chyba: project_root není definován. Spusťte prosím nejprve Buňku 1.")
    # Zde byste mohli buď vyvolat chybu, nebo znovu zavolat kód pro detekci project_root
    # Pro jednoduchost nyní předpokládáme, že project_root existuje.
    # V nejhorším případě nastavte fallback, ale to není ideální:
    # project_root = Path(".").resolve().parent # Velmi hrubý odhad, pokud CWD je notebooks

# Ujistěte se, že proměnná pro výstupní adresář je konzistentně pojmenována
# a definována pomocí project_root.
# Tuto definici byste měli mít v Buňce 1 a zde ji jen používat.
# Pro jistotu ji zde zopakujeme, pokud by se buňka spouštěla izolovaně
# a spoléhala na to, že project_root už je definován.
if 'output_dest_dir_for_attack' not in locals() and 'project_root' in locals():
     output_dest_dir_for_attack = project_root / "exports" / "notebook_attack_results"

# Určení délky a ID pro název souboru
# (Tato logika předpokládá, že ciphertext_file_path a ciphertext_to_crack jsou již definovány)
if ciphertext_file_path.exists() and ciphertext_file_path.is_file() and "text_1000_sample_1" in str(ciphertext_file_path.name):
    output_length = 1000
    output_sample_id = 1 
else: 
    output_length = len(ciphertext_to_crack) 
    output_sample_id = 99 

print(f"\nExporting results to directory: {output_dest_dir_for_attack}")
print(f"Using length={output_length}, sample_id={output_sample_id} for filenames.")

# Vytvoření adresáře, pokud neexistuje
output_dest_dir_for_attack.mkdir(parents=True, exist_ok=True)

export_result(
    plaintext=decrypted_plaintext,
    key=found_key,
    length=output_length, 
    sample_id=output_sample_id,
    dest=output_dest_dir_for_attack # Použijte konzistentní název proměnné
)

print("\nResults exported successfully. Check the following files:")
exported_plaintext_file = output_dest_dir_for_attack / f'text_{output_length}_sample_{output_sample_id}_plaintext.txt'
exported_key_file = output_dest_dir_for_attack / f'text_{output_length}_sample_{output_sample_id}_key.txt'

print(f"Plaintext: {exported_plaintext_file}")
print(f"Key:       {exported_key_file}")

# Kontrolní výpis obsahu exportovaného souboru
if exported_plaintext_file.exists():
    with open(exported_plaintext_file, 'r', encoding='utf-8') as f:
        print(f"\nContent of exported plaintext file (first 200 chars):\n{f.read(200)}...")
else:
    print(f"\nExported plaintext file not found at: {exported_plaintext_file}")


Exporting results to directory: c:\Git\DeShiftr\exports\notebook_attack_results
Using length=1000, sample_id=1 for filenames.

Results exported successfully. Check the following files:
Plaintext: c:\Git\DeShiftr\exports\notebook_attack_results\text_1000_sample_1_plaintext.txt
Key:       c:\Git\DeShiftr\exports\notebook_attack_results\text_1000_sample_1_key.txt

Content of exported plaintext file (first 200 chars):
AVOCEMADOASEYOALACLNAMSEAPECIADOABENTRAACLBYVEZLANEATLKAKUDRAVHFLVOASEPOAVZEVOATEDRAJEAKOSEBAHTLZANEATINEAHOKRVZLAYZLVOUATEDRAJEAKOSEBAOTEVFEZADVIFKLAVRNKOBIZACAVOCUALAHONTLVIZANEAHFEDAKOZLAJEDAFEKZAB...
